In [1]:
"""
CSIRO Image2Biomass - Tools for EDA, data quality checks, and model validation
"""

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Core ML & Stats
from sklearn.ensemble import IsolationForest, RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

PLOTLY_AVAILABLE = True

In [2]:
def load_and_pivot_biomass_data(csv_path):
    """
    Load CSIRO train.csv and convert from long to wide format.
    
    Args:
        csv_path: Path to train.csv
        
    Returns:
        DataFrame in wide format (1 row per image)
    """
    print("\nLoading CSIRO Biomass Dataset...")
    print("=" * 60)
    
    # Load data
    df_long = pd.read_csv(csv_path)
    print(f"Loaded: {len(df_long)} rows × {len(df_long.columns)} columns (long format)")
    
    # Extract image ID from sample_id
    df_long['image_id'] = df_long['sample_id'].str.split('__').str[0]
    
    # Pivot targets from long to wide
    df_wide = df_long.pivot_table(
        index=['image_id', 'image_path', 'Sampling_Date', 'State', 'Species', 
               'Pre_GSHH_NDVI', 'Height_Ave_cm'],
        columns='target_name',
        values='target',
        aggfunc='first'
    ).reset_index()
    
    print(f"Pivoted: {len(df_wide)} images × {len(df_wide.columns)} columns (wide format)")
    
    # Feature Engineering: Date features
    print("\nEngineering temporal features...")
    df_wide['Sampling_Date'] = pd.to_datetime(df_wide['Sampling_Date'], format='%m/%d/%Y', errors='coerce')
    df_wide['Month'] = df_wide['Sampling_Date'].dt.month
    df_wide['DayOfYear'] = df_wide['Sampling_Date'].dt.dayofyear
    df_wide['Season'] = pd.cut(df_wide['Month'], 
                                bins=[0, 3, 6, 9, 12], 
                                labels=['Summer', 'Autumn', 'Winter', 'Spring'])
    
    # Cyclical encoding for seasonality
    df_wide['Month_sin'] = np.sin(2 * np.pi * df_wide['Month'] / 12)
    df_wide['Month_cos'] = np.cos(2 * np.pi * df_wide['Month'] / 12)
    
    # Feature Engineering: Species complexity
    df_wide['Species_Count'] = df_wide['Species'].str.count('_') + 1
    df_wide['Has_Clover'] = df_wide['Species'].str.contains('Clover|clover', na=False).astype(int)
    df_wide['Has_Ryegrass'] = df_wide['Species'].str.contains('Ryegrass|ryegrass', na=False).astype(int)
    
    # Encode State as numeric
    le_state = LabelEncoder()
    df_wide['State_Encoded'] = le_state.fit_transform(df_wide['State'])
    
    # Derived biomass ratios (as mentioned in research)
    df_wide['Dead_Ratio'] = df_wide['Dry_Dead_g'] / (df_wide['Dry_Total_g'] + 1e-6)
    df_wide['Green_Ratio'] = df_wide['Dry_Green_g'] / (df_wide['Dry_Total_g'] + 1e-6)
    df_wide['Clover_Ratio'] = df_wide['Dry_Clover_g'] / (df_wide['Dry_Total_g'] + 1e-6)
    
    # NDVI-Height interaction (mentioned in research as strong predictor)
    df_wide['NDVI_Height_Interaction'] = df_wide['Pre_GSHH_NDVI'] * df_wide['Height_Ave_cm']
    
    # Biomass density
    df_wide['Biomass_per_Height'] = df_wide['Dry_Total_g'] / (df_wide['Height_Ave_cm'] + 1e-6)
    
    print(f"Final dataset: {len(df_wide)} samples x {len(df_wide.columns)} features")
    
    # Data quality report
    print("\nData Quality Summary:")
    print(f"  - Missing values: {df_wide.isnull().sum().sum()}")
    print(f"  - States: {df_wide['State'].unique().tolist()}")
    print(f"  - Date range: {df_wide['Sampling_Date'].min()} to {df_wide['Sampling_Date'].max()}")
    
    # Target statistics
    target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    print("\nTarget Statistics:")
    for col in target_cols:
        zero_pct = (df_wide[col] == 0).sum() / len(df_wide) * 100
        print(f"  - {col:15s}: mean={df_wide[col].mean():6.2f}g, "
              f"zeros={zero_pct:5.1f}%, CV={df_wide[col].std() / (df_wide[col].mean() + 1e-6):.2f}")
    
    return df_wide

In [3]:
class AnomalyDetector:
    """
    Multi-algorithm anomaly detection with ensemble scoring.
    Combines Isolation Forest, LOF, and statistical methods.
    """

    def __init__(self, contamination=0.05, n_neighbors=20):
        self.contamination = contamination
        self.n_neighbors = n_neighbors
        self.models = {}
        self.results = {}

    def detect_anomalies(self, df, numeric_cols):
        """
        Run multiple anomaly detection algorithms and ensemble results.
        """
        print("\nANOMALY DETECTION")
        print("=" * 60)

        X = df[numeric_cols].copy()
        X = X.fillna(X.median())

        # Standardize for distance-based methods
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Handle Zero Variance columns
        # StandardScaler creates NaNs if a column has 0 variance (division by zero) => replace these NaNs with 0
        X_scaled = np.nan_to_num(X_scaled)

        # 1. Isolation Forest (tree-based)
        print("Running Isolation Forest...")
        iso_forest = IsolationForest(
            contamination=self.contamination, 
            random_state=42, 
            n_estimators=200
        )
        iso_scores = iso_forest.fit_predict(X_scaled)
        iso_anomaly_scores = iso_forest.score_samples(X_scaled)
        
        # 2. Local Outlier Factor (density-based)
        print("Running Local Outlier Factor...")
        lof = LocalOutlierFactor(
            n_neighbors=self.n_neighbors, 
            contamination=self.contamination
        )
        lof_scores = lof.fit_predict(X_scaled)
        lof_anomaly_scores = lof.negative_outlier_factor_

        # 3. Statistical Z-score method (multivariate)
        print("Running Statistical Z-score...")
        z_scores = np.abs(stats.zscore(X_scaled, axis=0))
        z_anomalies = (z_scores > 3).any(axis=1).astype(int)
        z_anomalies = np.where(z_anomalies == 1, -1, 1)

        # -----------
        # Ensemble: Majority voting
        ensemble_votes = (iso_scores + lof_scores + z_anomalies) / 3
        is_anomaly = ensemble_votes < 0

        # Composite anomaly score (normalized)
        composite_score = (
            -iso_anomaly_scores / np.abs(iso_anomaly_scores).max() +
            -lof_anomaly_scores / np.abs(lof_anomaly_scores).max() +
            z_scores.max(axis=1) / 3
        ) / 3

        self.results = {
            'is_anomaly': is_anomaly,
            'anomaly_score': composite_score,
            'isolation_forest': iso_scores,
            'lof': lof_scores,
            'z_score': z_anomalies
        }
        
        # Report findings
        n_anomalies = is_anomaly.sum()
        print(f"\nDetected {n_anomalies} anomalies ({n_anomalies/len(df)*100:.2f}%)")
        
        if n_anomalies > 0:
            anomaly_indices = np.where(is_anomaly)[0]
            print(f"\nTop 5 anomaly indices: {anomaly_indices[:5].tolist()}")
            
            # Show which states have anomalies
            if 'State' in df.columns:
                anomaly_states = df.iloc[anomaly_indices]['State'].value_counts()
                print(f"\nAnomalies by State:")
                for state, count in anomaly_states.items():
                    print(f"  {state}: {count}")
        
        return self.results

In [4]:
class MultivariateAnalyzer:
    """
    Discovers interactions, correlations, and generates engineered features.
    """
    def __init__(self, correlation_threshold=0.7, interaction_threshold=0.3):
        self.correlation_threshold = correlation_threshold
        self.interaction_threshold = interaction_threshold
        self.interaction_features = []

    def analyze(self, df, numeric_cols, target_col=None):
        """
        Comprehensive multivariate analysis with automatic feature engineering.
        """
        print("\nMULTIVARIATE INTERACTION ANALYSIS")
        print("=" * 60)

        X = df[numeric_cols].copy()

        # 1. Correlation Analysis
        print("Computing correlation matrix...")
        corr_matrix = X.corr()

        # Find strong correlations
        strong_corr = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_val = corr_matrix.iloc[i, j]
                if abs(corr_val) > self.correlation_threshold:
                    strong_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_val))
        
        if strong_corr:
            print(f"\nFound {len(strong_corr)} strong correlations (|r| > {self.correlation_threshold}):")
            for col1, col2, corr in sorted(strong_corr, key=lambda x: abs(x[2]), reverse=True)[:15]:
                print(f"  {col1:25s} <-> {col2:25s}: r={corr:6.3f}")

        # 2. Automatic Interaction Feature Generation
        print("\nGenerating interaction features...")
        new_features = {}

        # Generate multiplicative and ratio features for correlated pairs
        for col1, col2, corr in strong_corr[:20]:
            # Multiplicative interaction
            interaction_name = f"{col1}_x_{col2}"
            new_features[interaction_name] = df[col1] * df[col2]

            # Ratio interaction (if no zeros)
            if (df[col2] != 0).all():
                ratio_name = f"{col1}_div_{col2}"
                new_features[ratio_name] = df[col1] / df[col2]

        # 3. Polynomial features for highly skewed columns
        for col in numeric_cols[:10]:
            if df[col].std() > 0:
                skew = df[col].skew()
                if abs(skew) > 1:
                    new_features[f"{col}_squared"] = df[col] ** 2
                    new_features[f"{col}_log"] = np.log1p(df[col].clip(lower=0))

        self.interaction_features = list(new_features.keys())
        print(f"Generated {len(new_features)} interaction features")

        # 4. If target provided, rank interactions by correlation with target
        if target_col and target_col in df.columns:
            print(f"\nRanking features by correlation with '{target_col}':")
            feature_df = pd.DataFrame(new_features)
            feature_corr = feature_df.corrwith(df[target_col]).abs().sort_values(ascending=False)
            
            print("Top 10 interaction features:")
            for feat, corr in feature_corr.head(10).items():
                print(f"  {feat:40s}: r={corr:.3f}")
        
        return new_features, corr_matrix

In [5]:
class ClusteringEngine:
    """
    Unsupervised pattern discovery with automatic cluster optimization.
    """
    
    def __init__(self, max_clusters=8):
        self.max_clusters = max_clusters
        self.best_model = None
        self.cluster_labels = None
        
    def discover_patterns(self, df, numeric_cols):
        """
        Automatic clustering with K-Means and DBSCAN, selecting optimal k.
        """
        print("\nUNSUPERVISED PATTERN DISCOVERY")
        print("=" * 60)
        
        X = df[numeric_cols].copy()
        X = X.fillna(X.median())

        
        # Standardize
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        X_scaled = np.nan_to_num(X_scaled)
        
        # 1. Determine optimal k using elbow method + silhouette
        print("Finding optimal number of clusters...")
        inertias = []
        silhouettes = []
        
        max_k = min(self.max_clusters + 1, len(X) // 10)
        
        for k in range(2, max_k):
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(X_scaled)
            inertias.append(kmeans.inertia_)
            silhouettes.append(silhouette_score(X_scaled, labels))

        # Find elbow using second derivative
        if len(silhouettes) > 2:
            inertia_diff = np.diff(inertias, 2)
            optimal_k = np.argmax(inertia_diff) + 2
        else:
            optimal_k = 2
                
        # Validate with silhouette
        sil_optimal_k = np.argmax(silhouettes) + 2
        
        # Use average of two methods
        final_k = int(np.mean([optimal_k, sil_optimal_k]))
        
        print(f"Optimal clusters: {final_k} (elbow: {optimal_k}, silhouette: {sil_optimal_k})")
        
        # 2. Fit final K-Means model
        kmeans = KMeans(
            n_clusters=final_k, 
            random_state=42, 
            n_init=20
        )
        kmeans_labels = kmeans.fit_predict(X_scaled)
        
        # 3. Try DBSCAN for density-based clustering
        print("\nAttempting DBSCAN for density-based patterns...")
        dbscan = DBSCAN(eps=0.5, min_samples=5)
        dbscan_labels = dbscan.fit_predict(X_scaled)
        n_dbscan_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
        
        print(f"DBSCAN found {n_dbscan_clusters} clusters + {(dbscan_labels == -1).sum()} noise points")
        
        # Use K-Means as primary, DBSCAN for validation
        self.cluster_labels = kmeans_labels
        self.best_model = kmeans
        
        # 4. Profile each cluster
        print(f"\nCLUSTER PROFILES (K={final_k}):")
        cluster_df = X.copy()
        cluster_df['Cluster'] = kmeans_labels
        
        for cluster_id in range(final_k):
            cluster_data = cluster_df[cluster_df['Cluster'] == cluster_id]
            size = len(cluster_data)
            pct = size / len(df) * 100
            
            print(f"\nCluster {cluster_id} ({size} samples, {pct:.1f}%):")
            
            # Find distinguishing features (highest z-score deviation from mean)
            cluster_mean = cluster_data[numeric_cols].mean()
            global_mean = X[numeric_cols].mean()
            global_std = X[numeric_cols].std()
            
            z_scores = (cluster_mean - global_mean) / (global_std + 1e-10)
            top_features = z_scores.abs().sort_values(ascending=False).head(3)
            
            for feat, z in top_features.items():
                direction = "higher" if z_scores[feat] > 0 else "lower"
                print(f"- {feat:25s}: {direction} than average (z={z:.2f})")
        
        return {
            'labels': kmeans_labels,
            'n_clusters': final_k,
            'silhouette': silhouettes[final_k - 2] if final_k - 2 < len(silhouettes) else silhouettes[-1],
            'dbscan_labels': dbscan_labels,
            'cluster_centers': kmeans.cluster_centers_
        }

In [6]:
class PredictiveSignalExtractor:
    """
    Feature importance analysis using ensemble methods.
    """
    
    def __init__(self, task_type='auto'):
        self.task_type = task_type
        self.model = None
        self.feature_importance = None
        
    def extract_signals(self, df, numeric_cols, target_col):
        """
        Use Random Forest to determine feature importance.
        """
        print("\nPREDICTIVE SIGNAL EXTRACTION")
        print("=" * 60)
        
        if target_col not in df.columns:
            print(f"Target column '{target_col}' not found")
            return None
        
        X = df[numeric_cols].copy()
        X = X.fillna(X.median())
        y = df[target_col].copy()
        
        # Remove target from features if present
        if target_col in X.columns:
            X = X.drop(columns=[target_col])
        
        # Auto-detect task type
        if self.task_type == 'auto':
            n_unique = y.nunique()
            task = 'classification' if n_unique <= 10 else 'regression'
        else:
            task = self.task_type
        
        print(f"Task type: {task.upper()}")
        print(f"Features: {len(X.columns)}, Samples: {len(X)}")
        
        # Train model
        if task == 'classification':
            model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
        else:
            model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
        
        model.fit(X, y)
        self.model = model
        
        # Extract feature importance
        importance_df = pd.DataFrame({
            'feature': X.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        self.feature_importance = importance_df
        
        # Calculate cumulative importance
        importance_df['cumulative'] = importance_df['importance'].cumsum()
        
        # Find features that contribute to 80% of importance
        n_features_80 = (importance_df['cumulative'] <= 0.8).sum()
        
        print(f"\nFEATURE IMPORTANCE RANKINGS:")
        print(f"Top {n_features_80} features explain 80% of predictive power\n")
        
        for idx, row in importance_df.head(15).iterrows():
            bar = '█' * int(row['importance'] * 50)
            print(f"{row['feature']:30s} {bar} {row['importance']:.4f}")
        
        # Identify low-importance features for potential removal
        low_importance = importance_df[importance_df['importance'] < 0.01]
        if len(low_importance) > 0:
            print(f"\n{len(low_importance)} features have <1% importance (consider removal)")
        
        return importance_df

In [7]:
class Visualizer:
    """
    Interactive and static visualizations.
    """
    
    def __init__(self, output_dir='insights_output'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)

    def plot_correlation_heatmap(self, corr_matrix, title="Correlation Heatmap"):
        plt.figure(figsize=(16, 14))
        
        sns.heatmap(corr_matrix, annot=False, fmt='.2f', cmap='RdBu_r', 
                    center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
        plt.title(title, fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        filename = self.output_dir / 'correlation_heatmap.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Saved: {filename}")
        plt.close()
        
    def plot_pairplot(self, df, numeric_cols, hue_col=None, max_vars=8):
        """High-dimensional pair plot with KDE."""
        plot_cols = numeric_cols[:max_vars]
        
        if hue_col and hue_col in df.columns:
            plot_df = df[plot_cols + [hue_col]].copy()
        else:
            plot_df = df[plot_cols].copy()
            hue_col = None
        
        g = sns.pairplot(plot_df, hue=hue_col, diag_kind='kde', plot_kws={'alpha': 0.6})
        g.fig.suptitle('Multivariate Pair Plot', y=1.02, fontsize=16, fontweight='bold')
        
        filename = self.output_dir / 'pairplot.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Saved: {filename}")
        plt.close()
        

    def plot_state_analysis(self, df, output_dir):
        """Biomass-specific: Analyze by State (WA anomaly)"""
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Dead biomass by state
        df.boxplot(column='Dry_Dead_g', by='State', ax=axes[0, 0])
        axes[0, 0].set_title('Dead Biomass by State')
        axes[0, 0].set_ylabel('Dry Dead (g)')
        
        # Clover by state
        df.boxplot(column='Dry_Clover_g', by='State', ax=axes[0, 1])
        axes[0, 1].set_title('Clover Biomass by State')
        axes[0, 1].set_ylabel('Dry Clover (g)')
        
        # Total biomass by state
        df.boxplot(column='Dry_Total_g', by='State', ax=axes[1, 0])
        axes[1, 0].set_title('Total Biomass by State')
        axes[1, 0].set_ylabel('Dry Total (g)')
        
        # NDVI by state
        df.boxplot(column='Pre_GSHH_NDVI', by='State', ax=axes[1, 1])
        axes[1, 1].set_title('NDVI by State')
        axes[1, 1].set_ylabel('NDVI')
        
        plt.suptitle('State-Level Analysis (WA Anomaly Investigation)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        filename = output_dir / 'state_analysis.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Saved: {filename}")
        plt.close()

In [8]:
class Biomass:
    """
    Main orchestrator
    """
    
    def __init__(self, output_dir='insights_output'):
        self.anomaly_detector = AnomalyDetector(contamination=0.05)
        self.multivariate_analyzer = MultivariateAnalyzer(correlation_threshold=0.6)
        self.clustering_engine = ClusteringEngine(max_clusters=8)
        self.signal_extractor = PredictiveSignalExtractor()
        self.visualizer = Visualizer(output_dir)
        self.insights = {}
        
    def analyze(self, df):
        """
        Run complete analysis pipeline.
        
        Args:
            df: Input DataFrame
            target_col: Target variable for supervised analysis
            id_cols: Columns to exclude from analysis (IDs, text, etc.)
        """
        print("\n" + "="*60)
        print("ANALYZING")
        print("="*60)
        
        # Identify numeric columns
        target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
        id_cols = ['image_id', 'image_path']
        
        # Numeric features for analysis
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        feature_cols = [col for col in numeric_cols if col not in target_cols]
        
        print(f"\nAnalysis Configuration:")
        print(f"  - Total samples: {len(df)}")
        print(f"  - Feature columns: {len(feature_cols)}")
        print(f"  - Target columns: {len(target_cols)}")
        
        # Module 1: Anomaly Detection
        anomaly_results = self.anomaly_detector.detect_anomalies(df, feature_cols)
        df['is_anomaly'] = anomaly_results['is_anomaly']
        df['anomaly_score'] = anomaly_results['anomaly_score']
        self.insights['anomalies'] = anomaly_results
        
        # Module 2: Multivariate Analysis (using Dry_Total_g as primary target)
        interaction_features, corr_matrix = self.multivariate_analyzer.analyze(
            df, feature_cols, target_col='Dry_Total_g'
        )
        self.insights['interactions'] = interaction_features
        self.insights['correlations'] = corr_matrix
        
        # Module 3: Clustering
        cluster_results = self.clustering_engine.discover_patterns(df, feature_cols)
        df['Cluster'] = cluster_results['labels']
        self.insights['clusters'] = cluster_results
        
        # Module 4: Predictive Signals
        print("\n" + "="*60)
        print("PREDICTIVE SIGNALS FOR EACH TARGET")
        print("="*60)
        
        importance_results = {}
        for target in target_cols:
            print(f"\n--- Analyzing: {target} ---")
            importance_df = self.signal_extractor.extract_signals(df, feature_cols, target)
            importance_results[target] = importance_df
        
        self.insights['feature_importance'] = importance_results
        
        # Module 5: Visualizations
        print("\nGENERATING VISUALIZATIONS")
        print("=" * 60)

        self.visualizer.plot_correlation_heatmap(corr_matrix, "Biomass Features Correlation Matrix")
        self.visualizer.plot_pairplot(df, feature_cols[:8], 
                                      hue_col='Cluster' if 'Cluster' not in feature_cols else None)
        self.visualizer.plot_state_analysis(df, self.visualizer.output_dir)

        # Save enhanced dataset
        output_csv = self.visualizer.output_dir / 'train_with_insights.csv'
        df.to_csv(output_csv, index=False)
        print(f"Saved enhanced dataset: {output_csv}")
        
        print("\n" + "="*60)
        print("ANALYSIS COMPLETE")
        print("="*60)
        print(f"\nAll outputs saved to: {self.visualizer.output_dir}")
        
        return df, self.insights
    
    def generate_biomass_report(self, df, output_file='biomass_report.txt'):
        """Generate a comprehensive text report of findings."""
        report_path = self.visualizer.output_dir / output_file
        
        with open(report_path, 'w') as f:
            f.write("="*70 + "\n")
            f.write("CSIRO BIOMASS COMPETITION - REPORT\n")
            f.write("="*70 + "\n\n")
            
            # Executive Summary
            f.write("EXECUTIVE SUMMARY\n")
            f.write("-"*70 + "\n")
            f.write(f"Dataset: {len(df)} pasture images analyzed\n")
            f.write(f"States: {df['State'].unique().tolist()}\n")
            f.write(f"Date Range: {df['Sampling_Date'].min()} to {df['Sampling_Date'].max()}\n\n")
            
            # Key Finding 1: WA Dead Matter Anomaly
            f.write("KEY FINDING 1: Western Australia Dead Matter Anomaly\n")
            f.write("-"*70 + "\n")
            wa_dead = df[df['State'] == 'WA']['Dry_Dead_g']
            other_dead = df[df['State'] != 'WA']['Dry_Dead_g']
            f.write(f"WA Dead Biomass:    mean={wa_dead.mean():.2f}g, zeros={(wa_dead==0).sum()/len(wa_dead)*100:.1f}%\n")
            f.write(f"Other Dead Biomass: mean={other_dead.mean():.2f}g, zeros={(other_dead==0).sum()/len(other_dead)*100:.1f}%\n")
            f.write("RECOMMENDATION: Apply state-conditional post-processing for Dead predictions\n\n")
            
            # Anomalies
            f.write("ANOMALY DETECTION RESULTS\n")
            f.write("-"*70 + "\n")
            if 'anomalies' in self.insights:
                n_anomalies = self.insights['anomalies']['is_anomaly'].sum()
                f.write(f"Detected: {n_anomalies} anomalous samples ({n_anomalies/len(df)*100:.2f}%)\n\n")
            
            # Clustering
            f.write("CLUSTER ANALYSIS\n")
            f.write("-"*70 + "\n")
            if 'clusters' in self.insights:
                f.write(f"Optimal clusters: {self.insights['clusters']['n_clusters']}\n")
                f.write(f"Silhouette score: {self.insights['clusters']['silhouette']:.3f}\n\n")
            
            # Feature Importance Summary
            f.write("TOP PREDICTIVE FEATURES (for Dry_Total_g)\n")
            f.write("-"*70 + "\n")
            if 'feature_importance' in self.insights:
                importance_df = self.insights['feature_importance']['Dry_Total_g']
                for _, row in importance_df.head(15).iterrows():
                    f.write(f"  {row['feature']:40s} {row['importance']:.4f}\n")
        
        print(f"Report saved: {report_path}")

In [9]:
def main():
    """Main execution function."""
    
    # Configuration
    TRAIN_CSV_PATH = '/kaggle/input/csiro-biomass/train.csv'
    OUTPUT_DIR = 'biomass_insights'
    
    print("CSIRO Biomass Competition - Insight Analysis")
    print("="*60)
    
    # Step 1: Load and preprocess data
    df_wide = load_and_pivot_biomass_data(TRAIN_CSV_PATH)
    
    # Step 2: Run comprehensive analysis
    engine = Biomass(output_dir=OUTPUT_DIR)
    df_enhanced, insights = engine.analyze(df_wide)
    
    # Step 3: Generate report
    engine.generate_biomass_report(df_enhanced)
    
    print("\nAnalysis Complete!")
    print(f"All outputs saved to: {OUTPUT_DIR}/")

if __name__ == "__main__":
    main()

CSIRO Biomass Competition - Insight Analysis

Loading CSIRO Biomass Dataset...
Loaded: 1785 rows × 9 columns (long format)
Pivoted: 357 images × 12 columns (wide format)

Engineering temporal features...
Final dataset: 357 samples x 26 features

Data Quality Summary:
  - Missing values: 2142
  - States: ['Tas', 'NSW', 'WA', 'Vic']
  - Date range: NaT to NaT

Target Statistics:
  - Dry_Green_g    : mean= 26.62g, zeros=  5.0%, CV=0.95
  - Dry_Dead_g     : mean= 12.04g, zeros= 11.2%, CV=1.03
  - Dry_Clover_g   : mean=  6.65g, zeros= 37.8%, CV=1.82
  - GDM_g          : mean= 33.27g, zeros=  0.0%, CV=0.75
  - Dry_Total_g    : mean= 45.32g, zeros=  0.0%, CV=0.62

ANALYZING

Analysis Configuration:
  - Total samples: 357
  - Feature columns: 15
  - Target columns: 5

ANOMALY DETECTION
Running Isolation Forest...
Running Local Outlier Factor...
Running Statistical Z-score...

Detected 19 anomalies (5.32%)

Top 5 anomaly indices: [7, 66, 92, 140, 147]

Anomalies by State:
  WA: 7
  Vic: 7
  NSW